In [79]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
p=[]
for dirname, _, filenames in os.walk('/kaggle/input'):
    filenames=sorted(filenames)
    for filename in filenames:
        p.append(os.path.join(dirname, filename))
print(p)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

['/kaggle/input/agi-comp/AGI/AGI.csv', '/kaggle/input/agi-comp/AGI/AGI tweets/2018-12-28.csv', '/kaggle/input/agi-comp/AGI/AGI tweets/2018-12-29.csv', '/kaggle/input/agi-comp/AGI/AGI tweets/2018-12-30.csv', '/kaggle/input/agi-comp/AGI/AGI tweets/2018-12-31.csv', '/kaggle/input/agi-comp/AGI/AGI tweets/2019-01-01.csv', '/kaggle/input/agi-comp/AGI/AGI tweets/2019-01-02.csv', '/kaggle/input/agi-comp/AGI/AGI tweets/2019-01-03.csv', '/kaggle/input/agi-comp/AGI/AGI tweets/2019-01-04.csv', '/kaggle/input/agi-comp/AGI/AGI tweets/2019-01-05.csv', '/kaggle/input/agi-comp/AGI/AGI tweets/2019-01-06.csv', '/kaggle/input/agi-comp/AGI/AGI tweets/2019-01-07.csv', '/kaggle/input/agi-comp/AGI/AGI tweets/2019-01-08.csv', '/kaggle/input/agi-comp/AGI/AGI tweets/2019-01-09.csv', '/kaggle/input/agi-comp/AGI/AGI tweets/2019-01-10.csv', '/kaggle/input/agi-comp/AGI/AGI tweets/2019-01-11.csv', '/kaggle/input/agi-comp/AGI/AGI tweets/2019-01-12.csv', '/kaggle/input/agi-comp/AGI/AGI tweets/2019-01-13.csv', '/kaggle/

IMPORTING LIBRARIES

In [38]:
import pandas as pd
import scipy
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [2]:
pip install mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.7/229.7 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 kB 14.6 MB/s eta 0:00:00
  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.9.0.post0
    Uninstalling python-dateutil-2.9.0.post0:
      Successfully uninstalled python-dateutil-2.9.0.post0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires cloudpickle~=2.2.1, but you have cloudpickle 3.0.0 which is incompatible.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 16.1.0 which is incompatible.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you ha

In [49]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model1 = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [69]:
import os
import time
from mistralai import Mistral
paths=[]
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        filenames=sorted(filenames)
        paths.append(os.path.join(dirname, filename))
print(paths)
n=3
all_days_img=[]
for i in range(1,n):
    path=paths[i]
    df=pd.read_csv(path)
    # print(df)
   
    # Retrieve the API key from environment variables (if needed)
    api_key = os.getenv('MISTRAL_API_KEY', '9GF8Qj0BYia55NLXS2FzzYn5HjAm9Hg9')
    
    # Specify model
    model = "pixtral-12b-2409"
    
    # Initialize the Mistral client
    client = Mistral(api_key=api_key)
    
    # Initialize an empty list to store responses
    text = []
    
    # Assuming 'photos' column contains lists of image URLs
    d1 = df['photos'].to_numpy()
    # c=0
    for i in list(d1):
        # print(i)
        if i=='[]':
            continue
        # Check if i is a list and extract the first image URL
        t=len(i)
        image_url=str(i[2:t-2])
        # print(image_url)
        
        # Define the messages for the chat
        messages = [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "EXPLAIN this image CLEARLY AND  strictly in one line with respect to financial sentiment analysis."
                    },
                    {
                        "type": "image_url",
                        "image_url": image_url
                    }
                ]
            }
        ]
    
        try:
            # Get the chat response
            chat_response = client.chat.complete(
                model=model,
                messages=messages
            )
    
            # Extract and print the content of the response
            response_text = chat_response.choices[0].message.content
            # print(response_text)
    
            # Append the response to the 'text' list
            text.append(response_text)
        
        except Exception as e:
            # Handle any exceptions that occur during the API request
            print(f"An error occurred: {e}")
        time.sleep(4)
    
    # Print the list of responses
    # print(text)
    preds_img = []
    total_neg_img=0
    total_pos_img=0
    total_neut_img=0
    preds_proba_img = []
    tokenizer_kwargs = {"padding": True, "truncation": True, "max_length": 512}
    for x in text:
        with torch.no_grad():
            input_sequence = tokenizer(x, return_tensors="pt", **tokenizer_kwargs)
            logits = model1(**input_sequence).logits
            scores_img = {
            k: v
            for k, v in zip(
                model1.config.id2label.values(),
                scipy.special.softmax(logits.numpy().squeeze()),
            )
        }
        sentimentFinbert = max(scores_img, key=scores_img.get)
        probabilityFinbert = max(scores_img.values())
        preds_img.append(sentimentFinbert)
        preds_proba_img.append(probabilityFinbert)
        total_neg_img+=scores_img['negative']
        total_pos_img+=scores_img['positive']
        total_neut_img+=scores_img['neutral']
        print(scores_img)
    total=len(text)
    mean_neg_img=total_neg_img/total
    mean_pos_img=total_pos_img/total
    mean_neut_img=total_neut_img/total
        
    print(mean_neg_img)
    print(mean_pos_img)
    print(mean_neut_img)
    d2={'negative':mean_neg_img,'positive':mean_pos_img,'neutral':mean_neut_img}
    all_days_img.append(d2)
    


['/kaggle/input/agi-comp/AGI/AGI.csv', '/kaggle/input/agi-comp/AGI/AGI tweets/2019-01-14.csv', '/kaggle/input/agi-comp/AGI/AGI tweets/2019-01-06.csv', '/kaggle/input/agi-comp/AGI/AGI tweets/2019-01-18.csv', '/kaggle/input/agi-comp/AGI/AGI tweets/2019-01-24.csv', '/kaggle/input/agi-comp/AGI/AGI tweets/2019-01-04.csv', '/kaggle/input/agi-comp/AGI/AGI tweets/2019-01-26.csv', '/kaggle/input/agi-comp/AGI/AGI tweets/2019-01-08.csv', '/kaggle/input/agi-comp/AGI/AGI tweets/2019-01-01.csv', '/kaggle/input/agi-comp/AGI/AGI tweets/2019-01-17.csv', '/kaggle/input/agi-comp/AGI/AGI tweets/2019-01-19.csv', '/kaggle/input/agi-comp/AGI/AGI tweets/2019-01-15.csv', '/kaggle/input/agi-comp/AGI/AGI tweets/2019-01-11.csv', '/kaggle/input/agi-comp/AGI/AGI tweets/2019-01-21.csv', '/kaggle/input/agi-comp/AGI/AGI tweets/2019-01-13.csv', '/kaggle/input/agi-comp/AGI/AGI tweets/2019-01-22.csv', '/kaggle/input/agi-comp/AGI/AGI tweets/2018-12-29.csv', '/kaggle/input/agi-comp/AGI/AGI tweets/2019-01-10.csv', '/kaggle/

In [70]:
print(all_days_img)

[{'negative': 0.1458570815855637, 'positive': 0.12889362089335918, 'neutral': 0.725249298568815}, {'negative': 0.33639195809761685, 'positive': 0.5024162869279584, 'neutral': 0.16119178105145693}]


In [71]:

import re
def clean_tweet(tweet):
    tweet = re.sub(r'http\S+|www\S+', '', tweet)   # Remove URLs
    tweet = re.sub(r'\$\w+', '', tweet)            # Remove cashtags
    tweet = re.sub(r'#\w+', '', tweet)             # Remove hashtags
    return tweet.strip()                           # Remove leading/trailing spaces

# Apply the function to the 'tweets' column
cleaned_tweets=[]
# print(unique)
for i in unique:
    # print(str(i))
    cleaned_tweets.append(clean_tweet(str(i)))
n=3
all_days_tweets=[]
for i in range(1,n):
    path=paths[i]
    df=pd.read_csv(path)
    unique=df['tweet'].unique()
    preds = []
    total_neg=0
    total_pos=0
    total_neut=0
    preds_proba = []
    tokenizer_kwargs = {"padding": True, "truncation": True, "max_length": 512}
    for x in cleaned_tweets:
        with torch.no_grad():
            input_sequence = tokenizer(x, return_tensors="pt", **tokenizer_kwargs)
            logits = model1(**input_sequence).logits
            scores = {
            k: v
            for k, v in zip(
                model1.config.id2label.values(),
                scipy.special.softmax(logits.numpy().squeeze()),
            )
        }
        sentimentFinbert = max(scores, key=scores.get)
        probabilityFinbert = max(scores.values())
        preds.append(sentimentFinbert)
        preds_proba.append(probabilityFinbert)
        total_neg+=scores['negative']
        total_pos+=scores['positive']
        total_neut+=scores['neutral']
        # print(scores)
    total=len(cleaned_tweets)
    mean_neg=total_neg/total
    mean_pos=total_pos/total
    mean_neut=total_neut/total
        
    print(mean_neg)
    print(mean_pos)
    print(mean_neut)
    d1={'negative':mean_neg,'positive':mean_pos,'neutral':mean_neut}
    all_days_tweets.append(d1)
        
        
        

0.06108293695182636
0.2685130311754243
0.670404034891519
0.06108293695182636
0.2685130311754243
0.670404034891519


In [72]:
print(all_days_tweets)

[{'negative': 0.06108293695182636, 'positive': 0.2685130311754243, 'neutral': 0.670404034891519}, {'negative': 0.06108293695182636, 'positive': 0.2685130311754243, 'neutral': 0.670404034891519}]


In [75]:
n=3
final_pred=[]
for i in range(n-1):
    print(i)
    negative_final=(all_days_tweets[i]['negative']+all_days_img[i]['negative'])/2
    positive_final=(all_days_tweets[i]['positive']+all_days_img[i]['positive'])/2
    neutral_final=(all_days_tweets[i]['neutral']+all_days_img[i]['neutral'])/2
    final_pred.append([negative_final,positive_final,neutral_final])
final_pred

0
1


[[0.10347000926869504, 0.19870332603439173, 0.697826666730167],
 [0.1987374475247216, 0.38546465905169136, 0.41579790797148797]]

In [80]:
df1=pd.DataFrame(final_pred,columns=['negative','positive','neutral'])
df1.to_csv('crypto_sentiment.csv',index=False)

**END**


**IMAGE TO TEXT CONVERSION** 

In [41]:
import os
import time
from mistralai import Mistral

# Retrieve the API key from environment variables (if needed)
api_key = os.getenv('MISTRAL_API_KEY', '9GF8Qj0BYia55NLXS2FzzYn5HjAm9Hg9')

# Specify model
model = "pixtral-12b-2409"

# Initialize the Mistral client
client = Mistral(api_key=api_key)

# Initialize an empty list to store responses
text = []

# Assuming 'photos' column contains lists of image URLs
d1 = df['photos'].to_numpy()
c=0
for i in list(d1):
    print(i)
    if i=='[]':
        continue
    # Check if i is a list and extract the first image URL
    t=len(i)
    print(c)
    image_url=str(i[2:t-2])
    # print(image_url)
    
    # Define the messages for the chat
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "EXPLAIN this image CLEARLY AND  strictly in one line with respect to financial sentiment analysis."
                },
                {
                    "type": "image_url",
                    "image_url": image_url
                }
            ]
        }
    ]

    try:
        # Get the chat response
        chat_response = client.chat.complete(
            model=model,
            messages=messages
        )

        # Extract and print the content of the response
        response_text = chat_response.choices[0].message.content
        print(response_text)

        # Append the response to the 'text' list
        text.append(response_text)
    
    except Exception as e:
        # Handle any exceptions that occur during the API request
        print(f"An error occurred: {e}")
    time.sleep(4)

# Print the list of responses
print(text)


['https://pbs.twimg.com/media/Dvrc1o0WsAINtR4.jpg']
0
The scatter plot illustrates the percent change in stock prices over 24 hours (x-axis) versus the percent change over 2 hours (y-axis), categorized into four sentiment groups: NSD, CRASH, REPO, and eDRNT.
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['https://pbs.twimg.com/media/DvrSyueXcAI-85q.jpg']
0
The image comprises three panels analyzing the correlation between Bitcoin (BTC) and two other assets, Russ2000 and Gold, over various time frames. The top panel displays a time series plot of the rolling 30-day correlation between BTC and the Russ2000 and Gold indices, showing varying correlations from positive to negative over the period from early 2018 to early 2019. The middle panel presents a scatter plot illustrating the relationship between daily returns of BTC and Russ2000, indicating a weak positive correlation (r=0.11). The bottom panel shows a scatter plot of BTC and Gold daily returns, indicating a moderate positive correlation (r=0.31).

TEXT FROM IMAGE

In [34]:
print(text)

['This scatter plot visualizes the relationship between the percent change in stock prices over 24 hours and the percent change over 2 hours, indicating financial sentiment analysis with labeled quadrants (NSD, CRASH, REPO, eDILNT).', 'The image presents a financial sentiment analysis showing the rolling 30-day correlation between Bitcoin (BTC) and two other assets: the Russian stock market (Russ2000) and gold. The top graph displays the rolling 30-day correlation, indicating fluctuating correlations over time with distinct periods of both positive and negative correlations. The middle graph illustrates the daily return percentages of BTC versus the daily returns of the Russ2000 index, highlighting a weak positive correlation (0.11). The bottom graph depicts the daily return percentages of BTC versus the daily returns of gold, showing a more moderate positive correlation (0.31). Overall, the graphs suggest varying degrees of correlation between BTC and the other assets, with gold exhib

In [46]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

IMAGE TEXT TO SENTIMENT

In [29]:
preds_img = []
total_neg_img=0
total_pos_img=0
total_neut_img=0
preds_proba_img = []
tokenizer_kwargs = {"padding": True, "truncation": True, "max_length": 512}
for x in text:
    with torch.no_grad():
        input_sequence = tokenizer(x, return_tensors="pt", **tokenizer_kwargs)
        logits = model(**input_sequence).logits
        scores_img = {
        k: v
        for k, v in zip(
            model.config.id2label.values(),
            scipy.special.softmax(logits.numpy().squeeze()),
        )
    }
    sentimentFinbert = max(scores_img, key=scores_img.get)
    probabilityFinbert = max(scores_img.values())
    preds_img.append(sentimentFinbert)
    preds_proba_img.append(probabilityFinbert)
    total_neg_img+=scores_img['negative']
    total_pos_img+=scores_img['positive']
    total_neut_img+=scores_img['neutral']
    print(scores_img)
total=len(text)
mean_neg_img=total_neg_img/total
mean_pos_img=total_pos_img/total
mean_neut_img=total_neut_img/total
    
print(mean_neg_img)
print(mean_pos_img)
print(mean_neut_img)
d2={'negative':mean_neg_img,'positive':mean_pos_img,'neutral':mean_neut_img}


{'positive': 0.021280406, 'negative': 0.035334785, 'neutral': 0.9433848}
{'positive': 0.039768975, 'negative': 0.029420916, 'neutral': 0.93081015}
{'positive': 0.057877023, 'negative': 0.085906796, 'neutral': 0.8562162}
{'positive': 0.027806416, 'negative': 0.93053365, 'neutral': 0.04165995}
{'positive': 0.02582366, 'negative': 0.025770042, 'neutral': 0.9484063}
{'positive': 0.02525505, 'negative': 0.02626313, 'neutral': 0.9484818}
{'positive': 0.30249232, 'negative': 0.007217938, 'neutral': 0.69028974}
{'positive': 0.81408733, 'negative': 0.006533563, 'neutral': 0.17937914}
{'positive': 0.8117012, 'negative': 0.0060847895, 'neutral': 0.18221402}
{'positive': 0.7624904, 'negative': 0.0073293513, 'neutral': 0.23018028}
0.11603949591517448
0.28885827530175445
0.5951022371649742


**TEXT TO SENTIMENT**

In [42]:

unique=df['tweet'].unique()


In [43]:
import re
def clean_tweet(tweet):
    tweet = re.sub(r'http\S+|www\S+', '', tweet)   # Remove URLs
    tweet = re.sub(r'\$\w+', '', tweet)            # Remove cashtags
    tweet = re.sub(r'#\w+', '', tweet)             # Remove hashtags
    return tweet.strip()                           # Remove leading/trailing spaces

# Apply the function to the 'tweets' column
cleaned_tweets=[]
# print(unique)
for i in unique:
    # print(str(i))
    cleaned_tweets.append(clean_tweet(str(i)))
# cleaned_tweets

    

TEXT TO SENTIMENT

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [47]:
preds = []
total_neg=0
total_pos=0
total_neut=0
preds_proba = []
tokenizer_kwargs = {"padding": True, "truncation": True, "max_length": 512}
for x in cleaned_tweets:
    with torch.no_grad():
        input_sequence = tokenizer(x, return_tensors="pt", **tokenizer_kwargs)
        logits = model(**input_sequence).logits
        scores = {
        k: v
        for k, v in zip(
            model.config.id2label.values(),
            scipy.special.softmax(logits.numpy().squeeze()),
        )
    }
    sentimentFinbert = max(scores, key=scores.get)
    probabilityFinbert = max(scores.values())
    preds.append(sentimentFinbert)
    preds_proba.append(probabilityFinbert)
    total_neg+=scores['negative']
    total_pos+=scores['positive']
    total_neut+=scores['neutral']
    # print(scores)
total=len(cleaned_tweets)
mean_neg=total_neg/total
mean_pos=total_pos/total
mean_neut=total_neut/total
    
print(mean_neg)
print(mean_pos)
print(mean_neut)
d1={'negative':mean_neg,'positive':mean_pos,'neutral':mean_neut}

{'positive': 0.04539643, 'negative': 0.8130698, 'neutral': 0.14153378}
{'positive': 0.027119199, 'negative': 0.72139084, 'neutral': 0.25149003}
{'positive': 0.035017293, 'negative': 0.38449988, 'neutral': 0.58048284}
{'positive': 0.0358605, 'negative': 0.38109356, 'neutral': 0.5830459}
{'positive': 0.031206498, 'negative': 0.62427074, 'neutral': 0.3445227}
{'positive': 0.038314503, 'negative': 0.3059852, 'neutral': 0.65570027}
{'positive': 0.03680514, 'negative': 0.27665147, 'neutral': 0.6865434}
{'positive': 0.03842455, 'negative': 0.26896805, 'neutral': 0.69260746}
{'positive': 0.039360486, 'negative': 0.2498732, 'neutral': 0.7107664}
{'positive': 0.03921418, 'negative': 0.28472593, 'neutral': 0.6760599}
{'positive': 0.03643918, 'negative': 0.33238485, 'neutral': 0.631176}
{'positive': 0.025242273, 'negative': 0.89187443, 'neutral': 0.082883246}
{'positive': 0.021363253, 'negative': 0.9402177, 'neutral': 0.038419027}
{'positive': 0.17674352, 'negative': 0.01279289, 'neutral': 0.81046

In [19]:
# preds_img

['neutral',
 'neutral',
 'neutral',
 'negative',
 'neutral',
 'neutral',
 'neutral',
 'positive',
 'positive',
 'positive']

**FINAL 1 DAYS SENTIMENT**

In [48]:
negative_final=(d1['negative']+d2['negative'])/2
positive_final=(d1['positive']+d2['positive'])/2
neutral_final=(d1['neutral']+d2['neutral'])/2
final_pred=[negative_final,positive_final,neutral_final]
final_pred

[0.13275400223210454, 0.3132978843801, 0.553948121510998]